In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import minimize
from scipy.stats import cauchy, norm, probplot, t, tvar

# Heavy-tailed data example

In [ ]:
data = np.array(
    [-26.8, -3.5, -3.4, -1.2,  0.4, 1.3, 2.3, 2.7, 3.0 , 3.2,  3.2,  3.5,
     3.6, 3.9, 4.2, 4.4, 5.0 , 6.5,  6.7,  7.1,  8.1, 10.5, 10.7, 24.0, 32.8]
)

In [ ]:
def get_aic_normal(data: np.ndarray) -> float:
    n = len(data)
    mu_hat = np.mean(data)
    sigma_sq_hat = tvar(data) * (n - 1) / n
    log_like_normal = norm.logpdf(data, loc=mu_hat, scale=np.sqrt(sigma_sq_hat))
    aic_normal = - 2 * np.sum(log_like_normal) + 4
    return aic_normal

In [ ]:
aic_normal = get_aic_normal(data)
print(f'normal AIC = {np.round(aic_normal, 1)}')

In [ ]:
def qq_plot(data: np.ndarray, title: str, ylim: tuple) -> None:
    probplot(data, dist='norm', fit=False, plot=plt)
    plt.title(title)
    plt.xlabel('Quantiles of standard normal')
    plt.ylabel('x')
    plt.ylim(ylim)
    plt.show()

In [ ]:
qq_plot(data, '(a) Radiation data', (-29, 35))

In [ ]:
def get_mu(data: np.ndarray) -> np.ndarray:
    n = len(data)
    mu_hat = np.mean(data)
    return np.linspace(
        mu_hat + 5 * np.std(data) / np.sqrt(n),
        mu_hat - 5 * np.std(data) / np.sqrt(n),
        num=100)


def profile_likelihood_normal(mu: np.ndarray, data: np.ndarray) -> np.ndarray:
    like = []
    for mu_ in mu:
        neg_log_like = profile_neg_log_likelihood_mu_normal(mu_, data)
        like.append(np.exp(-neg_log_like))
    like /= np.max(like)
    like = np.array(like)
    return like


def profile_neg_log_likelihood_mu_normal(mu: float, data: np.ndarray) -> float:
    n = len(data)
    sigma_mu_sq = (1 / n) * np.sum((data - mu)**2)
    like = sigma_mu_sq **(-n / 2)
    neg_log_like = -np.log(like)
    return neg_log_like


def profile_likelihood_cauchy(mu: np.ndarray, data: np.ndarray) -> np.ndarray:
    like = []
    x0 = 1.0
    for mu_ in mu:
        result = minimize(profile_neg_log_likelihood_cauchy, x0, args=(mu_,))
        neg_log_like = result.fun
        like.append(np.exp(-neg_log_like))
    like /= np.max(like)
    like = np.array(like)
    return like

def profile_neg_log_likelihood_cauchy(sigma: float, mu: float) -> float:
    return -np.sum(cauchy.logpdf(data, loc=mu, scale=sigma))


def neg_log_likelihood_mu_cauchy(x: np.ndarray) -> float:
    mu = x[0]
    sigma = x[1]
    return -np.sum(cauchy.logpdf(data, loc=mu, scale=sigma))


def profile_likelihood_t(mu: np.ndarray, data: np.ndarray, df: float) -> np.ndarray:
    like = []
    x0 = 1.0
    for mu_ in mu:
        result = minimize(profile_neg_log_likelihood_t, x0, args=(mu_, df))
        neg_log_like = result.fun
        like.append(np.exp(-neg_log_like))
    like /= np.max(like)
    like = np.array(like)
    return like

def profile_neg_log_likelihood_t(sigma: float, mu: float, df: float) -> float:
    return -np.sum(t.logpdf(data, loc=mu, scale=sigma, df=df))


def neg_log_likelihood_mu_t(x: np.ndarray, df: float) -> float:
    mu = x[0]
    sigma = x[1]
    return -np.sum(t.logpdf(data, loc=mu, scale=sigma, df=df))


def plot_profile_likelihoods(data: np.ndarray, t_dist_df: float):
    mu = get_mu(data)
    like_normal = profile_likelihood_normal(mu, data)
    like_cauchy = profile_likelihood_cauchy(mu, data)
    like_t = profile_likelihood_t(mu, data, df=t_dist_df)
    plt.plot(mu, like_normal, '--')
    plt.plot(mu, like_cauchy)
    plt.plot(mu, like_t, '-.')
    plt.axhline(y=0.15, linewidth=0.5)
    plt.xlabel(r'$\mu$')
    plt.ylabel('Likelihood')
    plt.title(r'(b) Profile likelihood of $\mu$')
    plt.legend(['normal', 'cauchy', 't'])

In [ ]:
def cauchy_aic(initial_guess: np.ndarray):
    result = minimize(neg_log_likelihood_mu_cauchy, initial_guess)
    sigma_hat = result.x[0]
    mu_hat = result.x[1]
    aic = 2 * result.fun + 4 
    print(f'Cauchy mu_hat = {np.round(sigma_hat, 1)}')
    print(f'Cauchy sigma_hat = {np.round(mu_hat, 1)}')
    print(f'Cauchy AIC = {np.round(aic, 1)}')

In [ ]:
x0=np.array([0.3, 1.0])
cauchy_aic(x0)

In [ ]:
def t_dist_aic(initial_guess: np.ndarray):
    nu = np.array([1, 2, 4, 8, 1e2, 1e3, 1e4])
    for nu_ in nu:
        result = minimize(neg_log_likelihood_mu_t, initial_guess, args=(nu_,))
        sigma_hat = result.x[0]
        mu_hat = result.x[1]
        aic = 2 * result.fun + 4
        print(f'nu = ', nu_)
        print(f't-distribution mu_hat = {np.round(sigma_hat, 1)}')
        print(f't-distribution sigma_hat = {np.round(mu_hat, 1)}')
        print(f't-distribution AIC = {np.round(aic, 1)}')
        print()

In [ ]:
t_dist_aic(x0)

In [ ]:
plot_profile_likelihoods(data, 1)

# Speed of Light Data From Example 4.8

In [ ]:
data = np.genfromtxt('../../R/LKPACK/michel.dat').flatten()
data = data[-40:]

In [ ]:
aic_normal = get_aic_normal(data)
print(f'normal AIC = {np.round(aic_normal, 1)}')

In [ ]:
qq_plot(data, '(c) Michelson data', (710, 960))

In [ ]:
cauchy_aic(x0)

In [ ]:
t_dist_aic(x0)

In [ ]:
plot_profile_likelihoods(data, 1e3)